In [2]:
import os
import pandas as pd
from tqdm import tqdm

In [3]:
imu_column_names = ['time(us)', 'gx(rad/s)', 'gy(rad/s)', 'gz(rad/s)', 'ax(m/s^2)', 'ay(m/s^2)', 'az(m/s^2)', 'temperature']
gt_column_names = ['time(us)', 'px', 'py', 'pz', 'qx', 'qy', 'qz', 'qw']

raw_data_folder = './../dat/raw/'
merged_data_folder = './../dat/merged/'

imu_file = '/imu_data.txt'
gt_file = '/mocap_data.txt'

In [4]:
for sequence_folder in tqdm(os.listdir(raw_data_folder), desc='Merging IMU and GT of Raw Sequences'):
    # Read IMU and GT data frames
    imu_df = pd.read_csv(raw_data_folder + sequence_folder + imu_file, sep=' ', header=None, names=imu_column_names, skiprows=1)
    gt_df = pd.read_csv(raw_data_folder + sequence_folder + gt_file, sep=' ', header=None, names=gt_column_names, skiprows=1)

    # Merge with nearest neighbor on time(us)
    merged_df = pd.merge_asof(gt_df, imu_df, on='time(us)', direction='nearest')

    # Reorder columns
    column_order = ['time(us)'] + imu_df.columns.tolist()[1:] + gt_df.columns.tolist()[1:]
    merged_df = merged_df[column_order] - merged

    # Separate start and end trajectories
    split_index = merged_df['time(us)'].diff().gt(1e6).idxmax()
    merged_df_start, merged_df_end = merged_df.iloc[:split_index], merged_df.iloc[split_index:]

    # Save merged data frames
    sequence_name = sequence_folder.split('_')[0]
    if len(merged_df_start) == 0:
        merged_df_end.to_csv(merged_data_folder + sequence_name + '_merged.txt', sep=' ', index=False, float_format='%.18e')
    elif len(merged_df_end) == 0:
        merged_df_start.to_csv(merged_data_folder + sequence_name + '-start_merged.txt', sep=' ', index=False, float_format='%.18e')
    else:
        merged_df_start.to_csv(merged_data_folder + sequence_name + '-start_merged.txt', sep=' ', index=False, float_format='%.18e')
        merged_df_end.to_csv(merged_data_folder + sequence_name + '-end_merged.txt', sep=' ', index=False, float_format='%.18e')

Merging IMU and GT of Raw Sequences: 100%|█████████████████████████████████████████████| 21/21 [00:13<00:00,  1.60it/s]
